# 读取数据

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
# 打开文件
file = open('./唐诗.txt', 'rb').read().decode(encoding='utf-8')
text = ''
# 逐行遍历
for line in file.split('\n'):
    try:
        # 跳过标题行
        int(line[0])
    except:
        # 添加非标题行
        text+=line
# 删除所有回车字符
text = text.replace('\r','')

In [3]:
text[:100]

'孤鸿海上来，池潢不敢顾。侧见双翠鸟，巢在三珠树。矫矫珍木巅，得无金丸惧。美服患人指，高明逼神恶。今我游冥冥，弋者何所慕。兰叶春葳蕤，桂华秋皎洁。欣欣此生意，自尔为佳节。谁知林栖者，闻风坐相悦。草木有本'

In [4]:
vocab = sorted(set(text))
print ('{}唯一字符'.format(len(vocab)))

2515唯一字符


获取数据映射

In [5]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = {i:u for i, u in enumerate(vocab)}
data = np.array([char2idx[c] for c in text])
dataset = tf.data.Dataset.from_tensor_slices(data)

In [8]:
for i in dataset.take(15):
    print(idx2char[i.numpy()], end='')

孤鸿海上来，池潢不敢顾。侧见双

In [9]:
# 分割句子
seq_length = 24
dataset = dataset.batch(seq_length+1, drop_remainder=True)
# 将batch分为模型输入和模型输出
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = dataset.map(split_input_target)
# 分割数据集，训练批次
dataset = dataset.shuffle(10000).batch(256, drop_remainder=True)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab), 1024,
                              batch_input_shape=[256, None]),
    tf.keras.layers.LSTM(1024,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(len(vocab))
  ])

In [13]:
model.compile(optimizer= tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['acc'])

D:\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
history = model.fit(dataset, epochs=5)


Epoch 1/5
3/3 [==============================] - 10s 3s/step - loss: 11.4505 - acc: 0.0137
Epoch 2/5
3/3 [==============================] - 9s 3s/step - loss: 9.2984 - acc: 0.0184
Epoch 3/5
3/3 [==============================] - 8s 3s/step - loss: 8.9953 - acc: 0.0084
Epoch 4/5
3/3 [==============================] - 9s 3s/step - loss: 8.8138 - acc: 0.0056
Epoch 5/5
3/3 [==============================] - 9s 3s/step - loss: 8.6457 - acc: 0.0195


In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
    print('\n模型输入：')
    for i in input_example_batch.numpy()[0]:
        print(idx2char[i], end='')
    print(f'\n\n模型预测：\n{idx2char[sampled_indices[0]]}',end='')


模型输入：
未更衣。银筝夜久殷勤弄，心怯空房不忍归！奉帚平明

模型预测：
博